<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/spark/netflix_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Netlix Pandas then Spark
Here we show you how it is easier to query data by converting it to Apache Spark.  There are limits.  For example, you cannot graph Spark data.  But you can study data and then later write code using Pandas once you figure out what data is important

First refer to this work by Diana:

https://colab.research.google.com/drive/1Nd7w840GkA3AuvVaEnirsjJzvH9M1Vvm#scrollTo=4Ud5EhU0dT7f


In [1]:
!wget https://raw.githubusercontent.com/werowe/HypatiaAcademy/refs/heads/master/spark/netflix_titles.csv

--2025-02-23 09:52:31--  https://raw.githubusercontent.com/werowe/HypatiaAcademy/refs/heads/master/spark/netflix_titles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3399671 (3.2M) [text/plain]
Saving to: ‘netflix_titles.csv’

netflix_titles.csv  100%[===================>]   3.24M  --.-KB/s    in 0.07s   

2025-02-23 09:52:32 (44.7 MB/s) - ‘netflix_titles.csv’ saved [3399671/3399671]



In [2]:


from pyspark.sql import SparkSession



# Initialize SparkSession
spark = SparkSession.builder \
    .appName("netflix") \
    .getOrCreate()

df = spark.read.csv(
    "netflix_titles.csv",
    header=True,        # Use the first row as column names
    inferSchema=True,   # Automatically infer data types
    sep=",",            # Specify delimiter (default is ',')
    encoding="UTF-8"    # Handle encoding
)

In [3]:

df.createOrReplaceTempView("netflix")

In [4]:
sql = '''
describe netflix

'''

result = spark.sql(sql)
result.show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|     show_id|   string|   NULL|
|        type|   string|   NULL|
|       title|   string|   NULL|
|    director|   string|   NULL|
|        cast|   string|   NULL|
|     country|   string|   NULL|
|  date_added|   string|   NULL|
|release_year|   string|   NULL|
|      rating|   string|   NULL|
|    duration|   string|   NULL|
|   listed_in|   string|   NULL|
| description|   string|   NULL|
+------------+---------+-------+



In [5]:

sql = '''
SELECT distinct country from netflix
'''

result = spark.sql(sql)
result.show()

+--------------------+
|             country|
+--------------------+
|Peru, United Stat...|
|India, United Kin...|
|Japan, Canada, Un...|
|United Kingdom, C...|
|      India, Germany|
|South Africa, Uni...|
|              Russia|
|United Kingdom, G...|
|Chile, United Sta...|
|South Africa, Angola|
|United States, Po...|
|  Philippines, Qatar|
|Hong Kong, China,...|
|  Germany, Sri Lanka|
|Denmark, France, ...|
|United States, Fr...|
|United Kingdom, N...|
|Australia, United...|
|Brazil, France, G...|
|United States, Ir...|
+--------------------+
only showing top 20 rows



In [6]:

sql = '''
SELECT count(country) AS country_count , country
FROM netflix
GROUP BY country
order by country_count desc

'''

result = spark.sql(sql)
result.show()

+-------------+--------------------+
|country_count|             country|
+-------------+--------------------+
|         2805|       United States|
|          972|               India|
|          419|      United Kingdom|
|          245|               Japan|
|          199|         South Korea|
|          181|              Canada|
|          145|               Spain|
|          123|              France|
|          110|              Mexico|
|          106|               Egypt|
|          105|              Turkey|
|           93|             Nigeria|
|           87|           Australia|
|           81|              Taiwan|
|           79|           Indonesia|
|           77|              Brazil|
|           75|         Philippines|
|           75|United Kingdom, U...|
|           73|United States, Ca...|
|           67|             Germany|
+-------------+--------------------+
only showing top 20 rows



In [7]:

sql = '''
select title, release_year
from netflix
order by release_year desc
'''

result = spark.sql(sql)
result.show()

+--------------------+-----------------+
|               title|     release_year|
+--------------------+-----------------+
| The Next Karate Kid|    United States|
|To All the Boys: ...|    June 12, 2021|
|What Would Sophia...| January 15, 2021|
| An Imperfect Murder| January 13, 2021|
|       The Challenge|December 15, 2020|
|        An Easy Girl|  August 13, 2020|
|                NULL|           40 min|
|   Dear White People|             2021|
|He-Man and the Ma...|             2021|
|       Blood & Water|             2021|
|           Intrusion|             2021|
|Jailbirds New Orl...|             2021|
|              Jaguar|             2021|
|       Midnight Mass|             2021|
|Monsters Inside: ...|             2021|
|The Great British...|             2021|
|My Heroes Were Co...|             2021|
|Vendetta: Truth, ...|             2021|
|Go! Go! Cory Cars...|             2021|
|        Je Suis Karl|             2021|
+--------------------+-----------------+
only showing top

In [9]:
sql = '''
select * from netflix
'''

result = spark.sql(sql)
result.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan